# Chatbot

In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Model

In [3]:
from langserve import RemoteRunnable

#os.environ["OPENAI_API_KEY"] = getpass.getpass()
#from langchain_openai import ChatOpenAI
#model = ChatOpenAI(model="gpt-3.5-turbo")
model = RemoteRunnable(os.getenv('LANGSERVE_ENDPOINT'))

## Message History

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello, Bob! I am an AI assistant. How can I help you today?', response_metadata={'model': 'llama-3-Korean-Bllossom-8B-gguf-Q4_K_M:latest', 'created_at': '2024-05-23T08:53:27.510054394Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 362669573, 'load_duration': 998004, 'prompt_eval_count': 55, 'prompt_eval_duration': 59296000, 'eval_count': 18, 'eval_duration': 161546000}, id='run-e9c734a6-824e-485f-9b5c-b1166fc888f1-0')

In [5]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content='Your name is not specified in our records. Please sign in to create a account or edit your existing account to add your name.', response_metadata={'model': 'llama-3-Korean-Bllossom-8B-gguf-Q4_K_M:latest', 'created_at': '2024-05-23T08:53:28.16457122Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 632428204, 'load_duration': 736317, 'prompt_eval_count': 15, 'prompt_eval_duration': 57823000, 'eval_count': 27, 'eval_duration': 433526000}, id='run-04aac89a-53fa-4153-9a5e-f6f1ad131b48-0')

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        #HumanMessage(content="Hi! My name is Bob"),
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob. Is there anything else I can help you with?\n', response_metadata={'model': 'llama-3-Korean-Bllossom-8B-gguf-Q4_K_M:latest', 'created_at': '2024-05-23T08:53:28.926977248Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 737387901, 'load_duration': 2184947, 'prompt_eval_count': 48, 'prompt_eval_duration': 133977000, 'eval_count': 18, 'eval_duration': 464207000}, id='run-8d4c3c16-4b00-4c07-a28a-9d3755bd3dd6-0')

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [8]:
# https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [9]:
config = {"configurable": {"session_id": "abc2"}}

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hello, Bob! I am an AI assistant. How can I help you today?'

In [11]:
store["abc2"]

InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Bob"), AIMessage(content='Hello, Bob! I am an AI assistant. How can I help you today?', response_metadata={'model': 'llama-3-Korean-Bllossom-8B-gguf-Q4_K_M:latest', 'created_at': '2024-05-23T08:53:29.30424132Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 309195945, 'load_duration': 970172, 'prompt_eval_duration': 10694000, 'eval_count': 18, 'eval_duration': 156234000}, id='run-819378be-e6d6-4fdf-91df-4637326a274a-0')])

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob. Is there anything else I can help you with?\n'

In [13]:
store["abc2"]

InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Bob"), AIMessage(content='Hello, Bob! I am an AI assistant. How can I help you today?', response_metadata={'model': 'llama-3-Korean-Bllossom-8B-gguf-Q4_K_M:latest', 'created_at': '2024-05-23T08:53:29.30424132Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 309195945, 'load_duration': 970172, 'prompt_eval_duration': 10694000, 'eval_count': 18, 'eval_duration': 156234000}, id='run-819378be-e6d6-4fdf-91df-4637326a274a-0'), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Bob. Is there anything else I can help you with?\n', response_metadata={'model': 'llama-3-Korean-Bllossom-8B-gguf-Q4_K_M:latest', 'created_at': '2024-05-23T08:53:29.878766317Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 352015566, 'load_duration': 1097543, 'prompt_eval_count': 22, 'prompt_eval_duration': 58851000, 'eval_count': 18, 'eval_duration': 156510000}

In [14]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is not specified in our records. Please sign in to create a account or edit your existing account to add your name.'

In [15]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your Name is Bob. I see your point. We can use your real name if you prefer. What would you like me to call you?\n'

In [16]:
store["abc2"].messages

[HumanMessage(content="Hi! I'm Bob"),
 AIMessage(content='Hello, Bob! I am an AI assistant. How can I help you today?', response_metadata={'model': 'llama-3-Korean-Bllossom-8B-gguf-Q4_K_M:latest', 'created_at': '2024-05-23T08:53:29.30424132Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 309195945, 'load_duration': 970172, 'prompt_eval_duration': 10694000, 'eval_count': 18, 'eval_duration': 156234000}, id='run-819378be-e6d6-4fdf-91df-4637326a274a-0'),
 HumanMessage(content="What's my name?"),
 AIMessage(content='Your name is Bob. Is there anything else I can help you with?\n', response_metadata={'model': 'llama-3-Korean-Bllossom-8B-gguf-Q4_K_M:latest', 'created_at': '2024-05-23T08:53:29.878766317Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 352015566, 'load_duration': 1097543, 'prompt_eval_count': 22, 'prompt_eval_duration': 58851000, 'eval_count': 18, 'eval_duration': 156510000}, id='run-9ee9ec86-b5e1-4e80-a991

In [17]:
#[f"{message.__class__.__name__.split()[-1]}:{message.content}" for message in store["abc2"].messages]

## Prompt templates

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [19]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Nice to meet you, Bob! I am an AI assistant. How can I help you today?'

In [20]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [21]:
config = {"configurable": {"session_id": "abc5"}}

In [22]:
response = with_message_history.invoke(
    #[HumanMessage(content="Hi! My name is Jim")],
    [HumanMessage(content="Hi! I'm Jim")],
    #{"messages":[HumanMessage(content="Hi! I'm Jim")]},
    config=config,
)

response.content

'Nice to meet you, Jim. I am an AI assistant. How can I help you today'

In [23]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    #{"messages": [HumanMessage(content="What's my name?")]},
    config=config,
)

response.content

'Your name is Jim. We just met, remember?\n'

In [24]:
[f"{message.__class__.__name__.split()[-1]}:{message.content}" for message in store["abc5"].messages]

["HumanMessage:Hi! I'm Jim",
 'AIMessage:Nice to meet you, Jim. I am an AI assistant. How can I help you today',
 "HumanMessage:What's my name?",
 'AIMessage:Your name is Jim. We just met, remember?\n']

In [25]:
store["abc5"].messages

[HumanMessage(content="Hi! I'm Jim"),
 AIMessage(content='Nice to meet you, Jim. I am an AI assistant. How can I help you today', response_metadata={'model': 'llama-3-Korean-Bllossom-8B-gguf-Q4_K_M:latest', 'created_at': '2024-05-23T08:53:31.840972218Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 381632241, 'load_duration': 1045542, 'prompt_eval_count': 16, 'prompt_eval_duration': 57764000, 'eval_count': 22, 'eval_duration': 189909000}, id='run-f8a6cbf9-f89d-43da-b96f-57b129655faf-0'),
 HumanMessage(content="What's my name?"),
 AIMessage(content='Your name is Jim. We just met, remember?\n', response_metadata={'model': 'llama-3-Korean-Bllossom-8B-gguf-Q4_K_M:latest', 'created_at': '2024-05-23T08:53:32.20044177Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 313455879, 'load_duration': 1034119, 'prompt_eval_count': 22, 'prompt_eval_duration': 58686000, 'eval_count': 14, 'eval_duration': 118981000}, id='run-083ffbad

In [26]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [27]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

"¡hola! Soy bien cercana asistente. Preguña me anything you want, and I'll do my best to answer it in Spanish.\n  ```\n "

## Managing Conversation History

In [28]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [29]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [30]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"it's a secret ;-) \n"

In [31]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "English",
    }
)
response.content

"that's a tough one. Do you have a favorite ice cream flavor?\n"

In [32]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [33]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"it's a secret ;-) \n"

In [34]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "English",
    },
    config=config,
)

response.content

"that's a good question! I think your favorite ice cream is the same as everybody elset: chocolate!"

## Streaming

In [35]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd. tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

Here| is| one|:| Why| was| the| math| book| so| sad|?| Because| it| had| too| many| prime| numbers||